# Bone Fracture Detection using YOLOv8

This Jupyter Notebook explores the application of deep learning for bone fracture detection using a comprehensive X-ray image dataset.  The dataset is specifically designed for computer vision projects and aims to facilitate the development and evaluation of automated bone fracture detection algorithms.

## About the Dataset

The dataset encompasses X-ray images categorized into several classes, each representing a specific type of bone fracture within the upper extremities. These classes include:

*   Elbow Positive
*   Fingers Positive
*   Forearm Fracture
*   Humerus Fracture
*   Shoulder Fracture
*   Wrist Positive

Each image is annotated with either bounding boxes or pixel-level segmentation masks, precisely indicating the location and extent of the detected fracture. These annotations are crucial for training and evaluating bone fracture detection algorithms, particularly object detection models.

This dataset provides a valuable resource for researchers and developers working on automated fracture detection. Its diverse range of fracture classes enables the training of robust models capable of accurately identifying fractures in various regions of the upper extremities. The ultimate goal of this dataset is to accelerate the development of computer vision solutions for automated fracture detection, thereby contributing to advancements in medical diagnostics and improved patient care.

**When using this dataset for your research, please cite it using the following DOI:** 10.13140/RG.2.2.14400.34569

**You can also find the dataset on ResearchGate:** [https://www.researchgate.net/publication/382268240_Bone_Fracture_Detection_Computer_Vision_Project](https://www.researchgate.net/publication/382268240_Bone_Fracture_Detection_Computer_Vision_Project)

## Imports

In [ ]:

from dotenv import load_dotenv

import tensorflow as tf
from tensorflow import keras
import cv2

tf.config.list_physical_devices('GPU'), tf.__version__

([PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')], '2.18.0')

## Download datasets

In [121]:
# Loading kaggle keys
load_dotenv()

True

In [122]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [ ]:
from hydra import initialize, compose

# https://gist.github.com/bdsaglam/586704a98336a0cf0a65a6e7c247d248

with initialize(version_base=None, config_path="conf"):
    cfg = compose(config_name="config")
    print(cfg.DATASET_DIRS.TRAIN_DIR)

datasets/BoneFractureYolo8/train/


## Loading Images

In [125]:
CLASS_NAMES = ['elbow positive',
               'fingers positive',
               'forearm fracture',
               'humerus fracture',
               'humerus',
               'shoulder fracture',
               'wrist positive']

class_mapping = dict(zip(range(len(CLASS_NAMES)), CLASS_NAMES))
class_mapping

{0: 'elbow positive',
 1: 'fingers positive',
 2: 'forearm fracture',
 3: 'humerus fracture',
 4: 'humerus',
 5: 'shoulder fracture',
 6: 'wrist positive'}

In [126]:
TRAIN_DIR = cfg.DATASET_DIRS.TRAIN_DIR
VALIDATION_DIR = cfg.DATASET_DIRS.VALIDATION_DIR
TEST_DIR = cfg.DATASET_DIRS.TEST_DIR

TRAIN_IMAGE_DIR = f'{TRAIN_DIR}/images'
TRAIN_LABELS_DIR = f'{TRAIN_DIR}/labels'

VALID_IMAGE_DIR = f'{VALIDATION_DIR}/images'
VALID_LABELS_DIR = f'{VALIDATION_DIR}/labels'

TEST_IMAGE = f'{TEST_DIR}/images'
TEST_LABELS = f'{TEST_DIR}/labels'

IMG_SIZE = cfg.TRAIN.IMG_SIZE
BATCH_SIZE = cfg.TRAIN.BATCH_SIZE

### Training Dataset setup

In [3]:
from utils.prepare_dataset import PrepareDataset


preparer_train_ds = PrepareDataset(image_dir=TRAIN_IMAGE_DIR,
                                   label_dir=TRAIN_LABELS_DIR)
image_paths, labels, bboxes = preparer_train_ds.get_dataset()
len(image_paths), len(labels), len(bboxes)

NameError: name 'TRAIN_IMAGE_DIR' is not defined

In [133]:
bboxes_tensor = tf.ragged.constant(bboxes)
classes_tensor = tf.ragged.constant(labels)
image_paths_tensor = tf.ragged.constant(image_paths)

train_data = tf.data.Dataset.from_tensor_slices(
    (image_paths_tensor, classes_tensor, bboxes_tensor))

In [134]:
# valid_datasets = PrepareDataset(image_dir=VALID_IMAGE_DIR,
#                                 label_dir=VALID_LABELS_DIR)

# valid_ds = tf.data.Dataset.from_tensor_slices((tf.ragged.constant(image_paths),
#                                                tf.ragged.constant(labels),
#                                                tf.ragged.constant(bboxes)))

In [138]:
def load_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    print(image.shape)
    return image


def load_dataset(image_path, classes, bbox):
    # Read Image
    image = load_image(image_path)
    bounding_boxes = {
        "classes": tf.cast(classes, dtype=tf.float32),
        "boxes": bbox,
    }
    return {"images": tf.cast(image, tf.float32), "bounding_boxes": bounding_boxes}

In [136]:
train_ds = train_data.map(load_dataset, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.shuffle(BATCH_SIZE * 4)
train_ds = train_ds.ragged_batch(BATCH_SIZE, drop_remainder=True)

(None, None, 3)


In [137]:
inputs = next(iter(train_ds.take(1)))
images, bounding_boxes = inputs["images"], inputs["bounding_boxes"]
image = images[0].numpy()
images[0].get_shape()

TensorShape([512, None, 3])